In [15]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import math 
import seaborn as sns

# Global variables
print("Configuration")
#path_root = "C:\\Users\\annaw\\Documents\\GitHub\\ideal_diet\\src\\"
path_root = "G:\\CIAT\\Code\\CWR\\ideal_diet\\src\\"
path_inputs = path_root +"inputs\\"
path_outputs = path_root + "outputs\\"

print("\tSetting parameters")
fao_element = [645,664]

print("\tLoading parameters")
groups = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='groups')
diets = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='diets_proportion')
years = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='years')
indicators_country = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='indicators_country')

print("\tOther things")
years_fao_colnames = ["Y"+str(y.year) for y in years.itertuples(index=True, name='Pandas')]

print("\tSuccessful")

Configuration
	Setting parameters
	Loading parameters
	Other things
	Successful


In [6]:
print("Merging fao data with groups")

print("\tLoading fao data")
data_fao = pd.read_csv(path_inputs + "faostat-food_supply.csv", encoding = "ISO-8859-1")

print("\tMerging fao data with groups")
data_fao_groups = pd.merge(data_fao, groups, left_on = "Item Code",right_on="Item Code",how='inner')

print("\tFiltering columns")
data_fao_colnames = ["ID","Area Code","Area","Item Code","Item_x","Element Code","Element","Unit","Description","Food group"]
data_fao_colnames.extend(years_fao_colnames)
data_fao_groups = data_fao_groups[data_fao_colnames]
data_fao_groups_colnames = ["id","area_code","area","item_code","item","element_code","element","unit","description","food_group"]
data_fao_groups_colnames.extend(years_fao_colnames)
data_fao_groups.columns = data_fao_groups_colnames

print("\tFiltering element " + str(fao_element))
data_fao_groups = data_fao_groups[data_fao_groups.element_code.isin(fao_element)]

print("\tPivoting food groups")
data_fao_groups = pd.pivot_table(data_fao_groups, values=years_fao_colnames, index=['area','food_group'], columns=['element_code'], aggfunc=np.sum,fill_value=0.0)    
data_fao_groups.reset_index(inplace=True)
data_fao_groups.columns = data_fao_groups.columns.droplevel()
data_fao_groups_colnames = ['area','food_group']
for y in years_fao_colnames:
    for e in fao_element:
        data_fao_groups_colnames.append(y + "_" + str(e))
data_fao_groups.columns = data_fao_groups_colnames

print("\tSaving")
data_fao_groups.to_excel(path_outputs +'1-data_fao_groups-proportion.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Merging fao data with groups
	Loading fao data
	Merging fao data with groups
	Filtering columns
	Filtering element [645, 664]
	Pivoting food groups
	Saving
	Successful


In [36]:
print("Merging fao data with diets and ssr")

print("\tLoading and merging SSR data")
# REMEMBER: You should execute ssr analysis before and put into inputs folder like csv file and just leave the columns
# with ssr results for each variable
ssr = pd.read_csv(path_inputs + "ssr.csv", encoding = "ISO-8859-1")
data_fao_diet = pd.merge(data_fao_groups, ssr, left_on = ["area","food_group"],right_on=["area","food_group"],how='inner')

# added a new year column name for comparing ssr analysis
#years_fao_colnames.append("ssr")    


prop = data_fao_diet.groupby(['area']).sum()

data_fao_diet = pd.merge(data_fao_diet, prop, left_on = "area",right_on="area",how='inner')

for y in years_fao_colnames:
    for e in fao_element:
        data_fao_diet[y+"_"+str(e)]=data_fao_diet[y+"_"+str(e)+"_x"]/data_fao_diet[y+"_"+str(e)+"_y"]


reg_exp = "^(" + "|".join(years_fao_colnames) + ")_([A-Za-z])*(\d)*_(" + "|".join(["x","y"]) + ")"
cols = data_fao_diet.columns.drop(list(data_fao_diet.filter(regex=reg_exp)))

data_fao_diet = data_fao_diet[cols]

print("\tMerging fao data with diet")
data_fao_diet = pd.merge(data_fao_diet, diets, left_on = "food_group",right_on="food_group",how='inner')

print("\tSaving")
data_fao_diet.to_excel(path_outputs +'2-data_fao_diet-proportion.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Merging fao data with diets and ssr
	Loading and merging SSR data
	Merging fao data with diet
	Saving
	Successful


                                      Y2013_645  Y2013_664      d664  d645  \
area                                                                         
Afghanistan                          952.093151     2095.0  1.116190   1.0   
Africa                              1599.309589     2600.2  1.116190   1.0   
Albania                             2897.446575     3113.8  1.116190   1.0   
Algeria                             2096.213699     3200.4  1.116190   1.0   
Americas                            2218.854795     3211.6  1.116190   1.0   
Angola                              1843.156164     2379.2  1.116190   1.0   
Antigua and Barbuda                 1887.589041     2337.6  1.116190   1.0   
Argentina                           2125.156164     3173.4  1.116190   1.0   
Armenia                             2555.857534     2834.0  1.116190   1.0   
Asia                                1738.509589     2745.6  1.116190   1.0   
Australia                           2554.263014     3234.4  1.11